In [1]:
!pip install flask-ngrok
! ngrok authtoken "225D7adRDbQhsVXdxHFBfdBy79f_6bjUDDY7kCxd79n1NkhCQ"
!pip install streamlit
!pip install torchxrayvision
!pip install pydicom
! pip install pyngrok
! pip install -U flask-cors

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [2]:
from flask_ngrok import run_with_ngrok
from flask_cors import CORS, cross_origin
from flask import Flask, request
import streamlit as st
from numpy import load
from numpy import expand_dims
from PIL import Image
import numpy as np
import os,sys
sys.path.insert(0,"..")
from glob import glob
import torch
import torchvision
import sys
import torch.nn.functional as F
import torchxrayvision as xrv
import pydicom as dicom
import pandas as pd
import matplotlib.pyplot as plt
import os
from skimage.transform import resize
import operator
from io import BytesIO
import torch
from PIL import Image
import base64
import cv2 as cv


In [3]:
# uploaded_file = st.file_uploader("Choose an X-Ray image to detect anomalies of the chest (the file must be a dicom extension or jpg)")

In [4]:
### Generate torchxrayvision model to find output probabilities       
def generatemodel(xrvmodel,wts):
    return xrvmodel(weights=wts)

In [5]:
### Read Chest X Ray Image
# def read_image(imgpath):

#     if (str(imgpath).find("jpg")!=-1) or (str(imgpath).find("png")!=-1):

#         # sample = Image.open("JPG_test/0c4eb1e1-b801903c-bcebe8a4-3da9cd3c-3b94a27c.jpg")
#           sample = Image.open(imgpath)
#           return np.array(sample)
#     if str(imgpath).find("dcm")!=-1:
#         img = dicom.dcmread(imgpath)
#         return img.pixel_array, img.PixelData

In [6]:
### Transform the image to ouput some illness
def transform2(img):
    input_tensor = torch.from_numpy(img).unsqueeze(0)
    img = input_tensor.numpy()[0, 0, :]
    img = (img / 1024.0 / 2.0) + 0.5
    img = np.clip(img, 0, 1)
    img = Image.fromarray(np.uint8(img * 255) , 'L')
    return img

In [7]:
### Transform the image to test an output image
def transform(img):

            img = ((img-img.min())/(img.max()-img.min())*255)
          
            img = xrv.datasets.normalize(np.array(img), 255) 
            
            # Check that images are 2D arrays
            if len(img.shape) > 2:
                img = img[:, :, 0]
            if len(img.shape) < 2:
                print("error, dimension lower than 2 for image")

            # Add color channel
            img = img[None, :, :]

            transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),
                                                        xrv.datasets.XRayResizer(224,engine="cv2")])

            img = transform(img)
            return img

In [8]:
### Returns the output probabilities of having certain illnesses anomalies
def testimage(model,img):
    # with torch.no_grad():
    model.eval()
    out = model(torch.from_numpy(img).unsqueeze(0)).cpu()    
    # out = model(img).cpu()   
    # out = torch.sigmoid(out)

    return {key:value  for (key,value)  in zip(model.pathologies, out.detach().numpy()[0]) if len(key)>2}

In [9]:
### Resize the model
def outputprob2(img,pr_model,visimage=True):
    ### Read an image
    img = resize(img, (img.shape[0] // 2, img.shape[1] // 2),
                    anti_aliasing=True)
       
    ### Preprocessmodel
    img_t = transform(img)
    ### Test an image
    return testimage(pr_model,img_t)

In [10]:
### Pipeline since we read an image until the ouput it is generated
def outputprob(imgpath,pr_model,visimage=True):
    ### Read an image
    img = read_image(imgpath)
    if visimage:
        plt.imshow(img,cmap="gray")
        plt.show()
    ### Preprocessmodel
    img_t = transform(img)
    ### Test an image
    return testimage(pr_model,img_t)

In [11]:
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'
run_with_ngrok(app) 
uploaded_file = st.file_uploader("Choose an X-Ray image to detect anomalies of the chest (the file must be a dicom extension or jpg)") 
def read_image(imgpath):

    if (str(imgpath).find("jpg")!=-1) or (str(imgpath).find("png")!=-1):

        # sample = Image.open("JPG_test/0c4eb1e1-b801903c-bcebe8a4-3da9cd3c-3b94a27c.jpg")
          sample = Image.open(imgpath)
          return np.array(sample)
    if str(imgpath).find("dcm")!=-1:
        # img = dicom.dcmread(imgpath)
        # return img.pixel_array, img.PixelData
        img = dicom.dcmread(imgpath)
        img=img.pixel_array
        return img
@app.route("/")
def index():
  return "Welcome to the X-Ray Disease Guesing App"
@app.route("/dicom",methods=['POST', 'GET'])
@cross_origin()
def get_img():
  org=bytes_in = request.args.get('dicom','').replace(" ","+")
  bytes_in = base64.b64encode(bytes(bytes_in, 'utf-8'))
  bytes_in1=base64.decodebytes(bytes_in) 
  with open('decoded_dicom.png', 'wb') as file_to_save:
      decoded_image_data = base64.decodebytes(bytes_in1)
      file_to_save.write(decoded_image_data)


  # file = open("decoded_image.png","rb")
  # bytes_in1 = file.read()
  # file.close()
  imgdef = read_image("decoded_dicom.png")
### Plot the input image

  fig, ax = plt.subplots()
  # img = cv.imread(imgdef)
  # print(img)
  ax.imshow(imgdef,cmap="gray")
  ax.figure.savefig("fig.png")
  st.pyplot(fig=fig)
  # Printing the possibility of having anomalies
  model = generatemodel(xrv.models.DenseNet,"densenet121-res224-mimic_ch") ### MIMIC MODEL+
  model.eval()
  pr = outputprob2(imgdef,model) 
  with open("fig.png","rb") as file:
    bytes_fig = file.read()
  bs64_fig = base64.b64encode(bytes_fig)
  bs64_fig=str(bs64_fig).replace(" ","+")[2:-1] #Base65
    
  # # Sort results by the descending probability order 
  pr = dict( sorted(pr.items(), key=operator.itemgetter(1),reverse=True))
  p=str(pr)
  # return bs64_fig, p
  # return str(pr)

  # return str(bs64_fig).replace(" ","+")[2:-1]
  # return f'<div style="display:flex; margin:auto"><img alt="" src=data:image/jpeg;base64,{bs64_fig} style="height:836px; width:592px; margin:1rem"><p>{p}</p></div>'
  return {"probability_data":p}

2022-01-12 05:46:02.971 
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
if __name__ == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2022-01-12 05:46:03.001  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8f96-34-80-7-10.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


2022-01-12 05:46:26.999 127.0.0.1 - - [12/Jan/2022 05:46:26] "GET / HTTP/1.1" 200 -
2022-01-12 05:47:04.051 127.0.0.1 - - [12/Jan/2022 05:47:04] "OPTIONS //dicom?dicom=... HTTP/1.1" 200 -


If this fails you can run `wget https://github.com/mlmed/torchxrayvision/releases/download/v1/mimic_ch-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt -O /root/.torchxrayvision/models_data/mimic_ch-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt`
[██████████████████████████████████████████████████]
Setting XRayResizer engine to cv2 could increase performance.


...

2022-01-12 05:48:21.588 127.0.0.1 - - [12/Jan/2022 05:48:21] "GET //dicom?dicom=/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxMTEhUTEhMWFRUXFxUVGBcYFxUVFRcVFRUXFhcVFRUYHSggGBolGxUVITEhJSkrLi4uFx8zODMtNygtLisBCgoKDg0OFxAQFy0dHR0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLf/AABEIAOgA2QMBIgACEQEDEQH/xAAcAAACAgMBAQAAAAAAAAAAAAAEBQMGAAIHAQj/xAA5EAABAwMCBAQFAwMEAQUAAAABAAIRAwQhBTESQVFhBnGBkSKhscHREzLwQlLhFBVi8XIWIzNDU//EABgBAQEBAQEAAAAAAAAAAAAAAAABAgME/8QAGxEBAQEBAQEBAQAAAAAAAAAAAAERAiESMVH/2gAMAwEAAhEDEQA/AOItBUzaRTyhpPVG09LC18p9K2y3PQoijaElWFmnBTNtIV+U+ienZQiBRA2CYtsi4gCZRVTTQ0xM9VrGdKBQW7KOdk5o6cT2HVGUdPa3fHc5PtyTDUGk2pdy/CsVK3gbJfTvKbesDnIWw16nyBKrOnNOmPJFW4MxAKVWmrUnnhBg9xCcWwAP4RBlVo4dhsl7qpOCJHRH1qYjc7Je+gzESOXWecoD7VoDZyhLh0/5RbfhZvKDNI7lAHWpSIhB17UxgSjbq5Y39zvRCDU6Z2n2RSq6bAVbvGyVeLgseIkT3/KR3WlZxg9D9ipY1KqVSzJOAT1hS0WYyOye06Ja7bzCOq6QKg4qO/NnPHTqpi/SqPtgeSErWIVnGnOmIM9I+ql/2Sf3EDsMphqj1LMhDvpkK/nQmdHHzMfRC3OjNH/0g+pU+V+lFIXistxpjf8A8iPLiQv+1/8AArPyv06VT0gDdrfWFKdMbzYI

Setting XRayResizer engine to cv2 could increase performance.


2022-01-12 05:48:31.314 127.0.0.1 - - [12/Jan/2022 05:48:31] "GET //dicom?dicom=... HTTP/1.1" 200 -
2022-01-12 05:48:39.690 127.0.0.1 - - [12/Jan/2022 05:48:39] "OPTIONS //dicom?dicom=... HTTP/1.1" 200 -


Setting XRayResizer engine to cv2 could increase performance.


2022-01-12 05:48:41.918 127.0.0.1 - - [12/Jan/2022 05:48:41] "GET //dicom?dicom=... HTTP/1.1" 200 -
